In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import time
from tensorflow.examples.tutorials.mnist import input_data
import warnings
warnings.filterwarnings('ignore')
mnist = input_data.read_data_sets('MNIST_data/', reshape=False, validation_size=0)
train_x = mnist.train.images

In [ ]:
total_epoch = 20
batch_size = 100
learning_rate = 0.0002
kernel_initializer = None

In [ ]:
def leakyrelu(x, a=0.2):
  return tf.maximum(a*x, x)

relu = tf.nn.relu

def random_noise(batch_size):
  return np.random.normal(0, 1, size=[batch_size, 1, 1, 100])

def generator(x, isTrain=True, reuse=False):
  with tf.variable_scope(name_or_scope='generator', reuse=reuse):

    conv1 = tf.layers.conv2d_transpose(x,filters=1024,kernel_size=4, strides=(1,1),padding='valid',kernel_initializer=kernel_initializer)
    relu1 = relu(tf.layers.batch_normalization(conv1,training=isTrain))

    conv2 = tf.layers.conv2d_transpose(relu1,filters=512,kernel_size=4, strides=(2,2),padding='same',kernel_initializer=kernel_initializer)
    relu2 = relu(tf.layers.batch_normalization(conv2,training=isTrain))

    conv3 = tf.layers.conv2d_transpose(relu2,filters=256,kernel_size=4, strides=(2,2),padding='same',kernel_initializer=kernel_initializer)
    relu3 = relu(tf.layers.batch_normalization(conv3,training=isTrain))

    conv4 = tf.layers.conv2d_transpose(relu3,filters=128,kernel_size=4, strides=(2,2),padding='same',kernel_initializer=kernel_initializer)
    relu4 = relu(tf.layers.batch_normalization(conv4,training=isTrain))

    conv5 = tf.layers.conv2d_transpose(relu4,filters=1,kernel_size=4, strides=(2,2),padding='same',kernel_initializer=kernel_initializer)
    output = tf.nn.tanh(conv5)

    return output

def discriminator(x, isTrain=True, reuse=False):
  with tf.variable_scope('discriminator', reuse=reuse):

    conv1 = tf.layers.conv2d(x,filters=128,kernel_size=4, strides=(2,2),padding='same',kernel_initializer=kernel_initializer)
    leakyrelu1 = leakyrelu(conv1)

    conv2 = tf.layers.conv2d(leakyrelu1,filters=256,kernel_size=4, strides=(2,2),padding='same',kernel_initializer=kernel_initializer)
    leakyrelu2 = leakyrelu(tf.layers.batch_normalization(conv2,training=isTrain))

    conv3 = tf.layers.conv2d(leakyrelu2,filters=512,kernel_size=4, strides=(2,2),padding='same',kernel_initializer=kernel_initializer)
    leakyrelu3 = leakyrelu(tf.layers.batch_normalization(conv3,training=isTrain))

    conv4 = tf.layers.conv2d(leakyrelu3,filters=1024,kernel_size=4, strides=(2,2),padding='same',kernel_initializer=kernel_initializer)
    leakyrelu4 = leakyrelu(tf.layers.batch_normalization(conv4,training=isTrain))

    conv5 = tf.layers.conv2d(leakyrelu4,filters=1,kernel_size=4, strides=(1,1),padding='valid',kernel_initializer=kernel_initializer)
    
    return conv5

In [ ]:
global_time0 = time.time()
g = tf.Graph()

with g.as_default():

  x = tf.placeholder(tf.float32, shape=(None,64,64,1))
  z = tf.placeholder(tf.float32, shape=(None,1,1,100))
  isTrain = tf.placeholder(dtype=tf.bool)

  G_z = generator(z)
  fake = discriminator(G_z, isTrain)
  real = discriminator(x, isTrain, reuse=True)

  D_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=real, labels=tf.ones([batch_size,1,1,1])))
  D_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=fake, labels=tf.zeros([batch_size,1,1,1])))  
  D_loss = D_loss_real + D_loss_fake
  G_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=fake, labels=tf.ones([batch_size, 1, 1, 1])))

  T_vars = tf.trainable_variables() # return list
  D_vars = [var for var in T_vars if var.name.startswith('discriminator')]
  G_vars = [var for var in T_vars if var.name.startswith('generator')]

  optimizer = tf.train.AdamOptimizer(learning_rate,beta1=.5)
  D_optim = optimizer.minimize(D_loss, var_list=D_vars)
  G_optim = optimizer.minimize(G_loss, var_list=G_vars)

In [ ]:
with tf.Session(graph=g) as sess:
  sess.run(tf.global_variables_initializer())
  total_batches = len(train_x) // batch_size
  sample_noise = random_noise(10)

  train_x = tf.image.resize_images(train_x, [64, 64]).eval()
  train_x = 2 * train_x - 1

  for epoch in range(total_epoch):
    epoch_time0 = time.time()
    
    for batch in range(total_batches):
      t0 = time.time()

      batch_x = train_x[batch*batch_size:(batch+1)*batch_size]
      noise = random_noise(batch_size)
      sess.run(D_optim, feed_dict={x : batch_x, z : noise, isTrain : True})
      noise = random_noise(batch_size)
      sess.run(G_optim, feed_dict={z : noise, isTrain : True})
      gl, dl = sess.run([G_loss, D_loss], feed_dict={x : batch_x, z : noise, isTrain : False})

      t1 = time.time()
      print('epoch : {:02d}/{:02d}'.format(epoch+1, total_epoch), end=' | ')
      print('batch : {:03d}/{:03d}'.format(batch+1, total_batches), end=' | ')
      print('time spent : {:.3f} sec'.format(t1-t0), end=' | ')
      print('gl : {:.6f} | dl : {:.6f}'.format(gl, dl))

      epoch_time1 = time.time()
      epoch_time = epoch_time1 - epoch_time0
      print('==================== EPOCH :', epoch+1, ' ended ====================')
      print('Generator Loss : {:.6f}\nDiscriminator Loss : {:.6f}'.format(gl, dl))
      print('Time Spent in This Epoch : {:.3f}'.format(epoch_time))

      generated = sess.run(G_z, feed_dict={z : sample_noise, isTrain : False}) / 2 + .5
      # generated = (generated + 1) * 255 / 2
      fig, ax = plt.subplots(1, 10, figsize=(10, 1))
      for i in range(10):
        ax[i].set_axis_off()
        ax[i].imshow(generated[i,:,:,0])
      plt.savefig('/gdrive/My Drive/Simple_DCGAN/{}.png'.format(str(epoch+1).zfill(3)), bbox_inches='tight')
      plt.close(fig)

  global_time1 = time.time()
  global_time_spent = global_time1 - global_time0
  print(('=' * 40 + '\n') * 3, 'Optimization Has Been Completed!!')
  print('Total Time Spent : {:.3f}'.format(global_time_spent))